In [168]:
import pandas as pd

In [169]:
latestmuni = pd.read_csv("2015medianmuni/ACS_15_5YR_B25077_with_ann.csv")
prevmuni = pd.read_csv("09medianbytown/ACS_09_5YR_B25077_with_ann.csv")
prevcounty=pd.read_csv("09counties/ACS_09_5YR_B25077_with_ann.csv")
latestcounty=pd.read_csv("2015mediancounties/ACS_15_5YR_B25077_with_ann.csv")

In [170]:
# Drop added row
latestmuni = latestmuni[latestmuni['GEO.id'] != 'Id']
prevmuni = prevmuni[prevmuni['GEO.id'] != 'Id']
latestcounty = latestcounty[latestcounty['GEO.id'] != 'Id']
prevcounty = prevcounty[prevcounty['GEO.id'] != 'Id']

In [171]:
latestframes= [latestmuni, latestcounty]
prevframes=[prevmuni, prevcounty]
latestmuni = pd.concat(latestframes, ignore_index=True)
prevmuni = pd.concat(prevframes, ignore_index=True)

In [172]:
latestmuni = latestmuni.rename(columns={ # yes, we could have done this when we read in the CSV
        'HD01_VD01': '2015value',
        'HD02_VD01': '2015valuemoe',
    })
prevmuni = prevmuni.rename(columns={ # yes, we could have done this when we read in the CSV
        'HD01_VD01': '2009valuewoinf',
        'HD02_VD01': '2009valuemoewoinf',
    })

In [173]:
# Remove "new jersey" label
latestmuni['GEOdisplaylabel'] = latestmuni['GEO.display-label'].replace(to_replace=', New Jersey', value='', regex=True)
prevmuni['GEOdisplaylabel'] = prevmuni['GEO.display-label'].replace(to_replace=', New Jersey', value='', regex=True)

In [174]:
latestmuni.head()

,GEO.id,GEO.id2,GEO.display-label,2015value,2015valuemoe,GEOdisplaylabel
0,0600000US3400100000,3400100000,"County subdivisions not defined, Atlantic Coun...",-,**,"County subdivisions not defined, Atlantic County"
1,0600000US3400100100,3400100100,"Absecon city, Atlantic County, New Jersey",206900,16458,"Absecon city, Atlantic County"
2,0600000US3400102080,3400102080,"Atlantic City city, Atlantic County, New Jersey",198200,15644,"Atlantic City city, Atlantic County"
3,0600000US3400107810,3400107810,"Brigantine city, Atlantic County, New Jersey",364000,15718,"Brigantine city, Atlantic County"
4,0600000US3400108680,3400108680,"Buena borough, Atlantic County, New Jersey",187500,10554,"Buena borough, Atlantic County"


In [175]:
# Split town and county names
latest_newcols = pd.DataFrame(latestmuni.GEOdisplaylabel.str.split(',',1).tolist(), columns = ['GEOdisplaylabel', 'County'])
prev_newcols = pd.DataFrame(prevmuni.GEOdisplaylabel.str.split(',',1).tolist(), columns = ['GEOdisplaylabel', 'County'])

In [176]:
latestmuni = pd.concat([latest_newcols, latestmuni], axis=1)
prevmuni = pd.concat([prev_newcols, prevmuni], axis=1)

In [177]:
# Remove county label
latestmuni['County'] = latestmuni['County'].replace(to_replace='County', value='', regex=True)
prevmuni['County'] = prevmuni['County'].replace(to_replace='County', value='', regex=True)


In [178]:
muni_merge = pd.merge(latestmuni, prevmuni, left_on='GEO.id2', right_on='GEO.id2', how='left')

In [179]:
muni_merge.head()

,GEOdisplaylabel_x,County_x,GEO.id_x,GEO.id2,GEO.display-label_x,2015value,2015valuemoe,GEOdisplaylabel_x,GEOdisplaylabel_y,County_y,GEO.id_y,GEO.display-label_y,2009valuewoinf,2009valuemoewoinf,GEOdisplaylabel_y
0,County subdivisions not defined,Atlantic,0600000US3400100000,3400100000,"County subdivisions not defined, Atlantic Coun...",-,**,"County subdivisions not defined, Atlantic County",County subdivisions not defined,Atlantic,0600000US3400100000,"County subdivisions not defined, Atlantic Coun...",-,**,"County subdivisions not defined, Atlantic County"
1,Absecon city,Atlantic,0600000US3400100100,3400100100,"Absecon city, Atlantic County, New Jersey",206900,16458,"Absecon city, Atlantic County",Absecon city,Atlantic,0600000US3400100100,"Absecon city, Atlantic County, New Jersey",259100,17220,"Absecon city, Atlantic County"
2,Atlantic City city,Atlantic,0600000US3400102080,3400102080,"Atlantic City city, Atlantic County, New Jersey",198200,15644,"Atlantic City city, Atlantic County",Atlantic City city,Atlantic,0600000US3400102080,"Atlantic City city, Atlantic County, New Jersey",231100,10782,"Atlantic City city, Atlantic County"
3,Brigantine city,Atlantic,0600000US3400107810,3400107810,"Brigantine city, Atlantic County, New Jersey",364000,15718,"Brigantine city, Atlantic County",Brigantine city,Atlantic,0600000US3400107810,"Brigantine city, Atlantic County, New Jersey",448200,18508,"Brigantine city, Atlantic County"
4,Buena borough,Atlantic,0600000US3400108680,3400108680,"Buena borough, Atlantic County, New Jersey",187500,10554,"Buena borough, Atlantic County",Buena borough,Atlantic,0600000US3400108680,"Buena borough, Atlantic County, New Jersey",199800,25289,"Buena borough, Atlantic County"


In [180]:
muni_merge = muni_merge.drop(["GEOdisplaylabel_y",'GEO.display-label_y', 'County_y', 'GEO.id_y'], axis=1)
muni_merge.rename(columns={'GEOdisplaylabel_x': 'GEOdisplaylabel', 'County_x': 'County', 'GEO.id_x': 'GEO.id', 'GEO.display-label_x': 'GEOdisplaylabel_long'}, inplace=True)

In [181]:
muni_merge.dtypes

GEOdisplaylabel         object
County                  object
GEO.id                  object
GEO.id2                 object
GEOdisplaylabel_long    object
2015value               object
2015valuemoe            object
GEOdisplaylabel         object
2009valuewoinf          object
2009valuemoewoinf       object
dtype: object

In [182]:
muni_merge['2009_median_value_temp'] = pd.to_numeric(muni_merge['2009valuewoinf'], errors='coerce') * (.100469602)
muni_merge["2009valuewoinf"] = pd.to_numeric(muni_merge["2009valuewoinf"], errors="coerce")

In [183]:
muni_merge["2009_median_value"] = muni_merge["2009valuewoinf"] + muni_merge["2009_median_value_temp"]

In [184]:
muni_merge['2009_moe_value_temp'] = pd.to_numeric(muni_merge['2009valuemoewoinf'], errors='coerce') * (.100469602)
muni_merge["2009valuemoewoinf"] = pd.to_numeric(muni_merge["2009valuemoewoinf"], errors="coerce")

In [185]:
muni_merge["2009_median_value_moe"] = muni_merge["2009valuemoewoinf"] + muni_merge["2009_moe_value_temp"]

In [186]:
muni_merge = muni_merge.drop(['2009_median_value_temp', '2009_moe_value_temp'], axis=1)

In [187]:
muni_merge.head()

,GEOdisplaylabel,County,GEO.id,GEO.id2,GEOdisplaylabel_long,2015value,2015valuemoe,GEOdisplaylabel,2009valuewoinf,2009valuemoewoinf,2009_median_value,2009_median_value_moe
0,County subdivisions not defined,Atlantic,0600000US3400100000,3400100000,"County subdivisions not defined, Atlantic Coun...",-,**,"County subdivisions not defined, Atlantic County",NaN,NaN,NaN,NaN
1,Absecon city,Atlantic,0600000US3400100100,3400100100,"Absecon city, Atlantic County, New Jersey",206900,16458,"Absecon city, Atlantic County",259100.0,17220.0,285131.673878,18950.086546
2,Atlantic City city,Atlantic,0600000US3400102080,3400102080,"Atlantic City city, Atlantic County, New Jersey",198200,15644,"Atlantic City city, Atlantic County",231100.0,10782.0,254318.525022,11865.263249
3,Brigantine city,Atlantic,0600000US3400107810,3400107810,"Brigantine city, Atlantic County, New Jersey",364000,15718,"Brigantine city, Atlantic County",448200.0,18508.0,493230.475616,20367.491394
4,Buena borough,Atlantic,0600000US3400108680,3400108680,"Buena borough, Atlantic County, New Jersey",187500,10554,"Buena borough, Atlantic County",199800.0,25289.0,219873.826480,27829.775765


In [190]:
print(356800 * .100469602) + 356800
print (185400 * .100469602) + 185400

392647.553994
204027.064211


In [193]:
muni_merge["2015valuemoeper"] = pd.to_numeric(muni_merge["2015valuemoe"],errors="coerce")/ pd.to_numeric(muni_merge["2015value"], errors="coerce")
muni_merge["2009valuemoeper"] = pd.to_numeric(muni_merge["2009_median_value_moe"],errors="coerce") / pd.to_numeric(muni_merge["2009_median_value"], errors="coerce")

In [194]:
muni_merge.head()

,GEOdisplaylabel,County,GEO.id,GEO.id2,GEOdisplaylabel_long,2015value,2015valuemoe,GEOdisplaylabel,2009valuewoinf,2009valuemoewoinf,2009_median_value,2009_median_value_moe,nj_latest,nj_previous,us_latest,us_previous,2015valuemoeper,2009valuemoeper
0,County subdivisions not defined,Atlantic,0600000US3400100000,3400100000,"County subdivisions not defined, Atlantic Coun...",-,**,"County subdivisions not defined, Atlantic County",NaN,NaN,NaN,NaN,315900,392647.553994,178600,204027.064211,NaN,NaN
1,Absecon city,Atlantic,0600000US3400100100,3400100100,"Absecon city, Atlantic County, New Jersey",206900,16458,"Absecon city, Atlantic County",259100.0,17220.0,285131.673878,18950.086546,315900,392647.553994,178600,204027.064211,0.079546,0.066461
2,Atlantic City city,Atlantic,0600000US3400102080,3400102080,"Atlantic City city, Atlantic County, New Jersey",198200,15644,"Atlantic City city, Atlantic County",231100.0,10782.0,254318.525022,11865.263249,315900,392647.553994,178600,204027.064211,0.078930,0.046655
3,Brigantine city,Atlantic,0600000US3400107810,3400107810,"Brigantine city, Atlantic County, New Jersey",364000,15718,"Brigantine city, Atlantic County",448200.0,18508.0,493230.475616,20367.491394,315900,392647.553994,178600,204027.064211,0.043181,0.041294
4,Buena borough,Atlantic,0600000US3400108680,3400108680,"Buena borough, Atlantic County, New Jersey",187500,10554,"Buena borough, Atlantic County",199800.0,25289.0,219873.826480,27829.775765,315900,392647.553994,178600,204027.064211,0.056288,0.126572


In [191]:
muni_merge["nj_latest"] = 315900
muni_merge["nj_previous"] = 392647.5539936
muni_merge["us_latest"] = 178600
muni_merge["us_previous"] = 204027.064211

In [203]:
muni_merge['value_change'] = ((pd.to_numeric(muni_merge['2015value'], errors='coerce') - muni_merge['2009_median_value'])/muni_merge['2009_median_value'])*100
muni_merge['nj_value_change'] = ((pd.to_numeric(muni_merge['nj_latest'], errors='coerce') - muni_merge['nj_previous'])/muni_merge['nj_previous'])*100
muni_merge['us_value_change'] = ((muni_merge['us_latest']-muni_merge['us_previous'])/muni_merge['us_previous'])*100


In [204]:
muni_merge.head()

,GEOdisplaylabel,County,GEO.id,GEO.id2,GEOdisplaylabel_long,2015value,2015valuemoe,GEOdisplaylabel,2009valuewoinf,2009valuemoewoinf,...,2009_median_value_moe,nj_latest,nj_previous,us_latest,us_previous,2015valuemoeper,2009valuemoeper,value_change,nj_value_change,us_value_change
0,County subdivisions not defined,Atlantic,0600000US3400100000,3400100000,"County subdivisions not defined, Atlantic Coun...",-,**,"County subdivisions not defined, Atlantic County",NaN,NaN,...,NaN,315900,392647.553994,178600,204027.064211,NaN,NaN,NaN,-19.546169,-12.462594
1,Absecon city,Atlantic,0600000US3400100100,3400100100,"Absecon city, Atlantic County, New Jersey",206900,16458,"Absecon city, Atlantic County",259100.0,17220.0,...,18950.086546,315900,392647.553994,178600,204027.064211,0.079546,0.066461,-27.437034,-19.546169,-12.462594
2,Atlantic City city,Atlantic,0600000US3400102080,3400102080,"Atlantic City city, Atlantic County, New Jersey",198200,15644,"Atlantic City city, Atlantic County",231100.0,10782.0,...,11865.263249,315900,392647.553994,178600,204027.064211,0.078930,0.046655,-22.066236,-19.546169,-12.462594
3,Brigantine city,Atlantic,0600000US3400107810,3400107810,"Brigantine city, Atlantic County, New Jersey",364000,15718,"Brigantine city, Atlantic County",448200.0,18508.0,...,20367.491394,315900,392647.553994,178600,204027.064211,0.043181,0.041294,-26.200829,-19.546169,-12.462594
4,Buena borough,Atlantic,0600000US3400108680,3400108680,"Buena borough, Atlantic County, New Jersey",187500,10554,"Buena borough, Atlantic County",199800.0,25289.0,...,27829.775765,315900,392647.553994,178600,204027.064211,0.056288,0.126572,-14.723820,-19.546169,-12.462594


In [206]:
muni_merge = muni_merge[muni_merge['GEOdisplaylabel'] != "County subdivisions not defined"]

ValueError: cannot reindex from a duplicate axis

In [205]:
muni_merge.to_csv("housingvalues.csv")